## Part 3
*Notice*: Starts divide the dataset into traning and testing set.

In [1]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(college, test_size=0.2, random_state=42)

NameError: name 'college' is not defined

### Prepare the data for MAchine Learning algorithms

We create a new copy of the train_set and separate the predictors and labels. The DataFrame college contains the predictors and the series college_labels consists of the variable PCTFLOAN. We confirm their dimensions.

In [305]:
college = train_set.drop("PCTFLOAN", axis=1)
college_labels = train_set["PCTFLOAN"].copy()
print(college.shape)
print(college_labels.shape)

(6162, 6)
(6162,)


*Notice* This part of code is actullay used at the last step of data cleaning.

### Data cleaning

Most Machine Learning algorithms cannot work with missing features, so let's create a few functions to take care of them.

Here we could set the missing values to the median of the attributes.

*Notice*: Because we also have missing values in our dependent varible, we need to clean the train set entirely.

In [306]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy="median")

Since the median can only be computed on numeriacal attributes, we need to create a copy of the data withou the text attributes. For our dataset, since there is no text attributes, we don't neet to do it.

Fit the imputer instance to the training data using the fit() method:

In [307]:
imputer.fit(train_set)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

The imputer has compited the median of each attribute and stored the result in its **statistics_** instance varible

In [308]:
imputer.statistics_

array([  5.86450000e-01,   9.50000000e+03,   2.79107766e+04,
         2.00000000e+00,   5.00000000e+00,   3.88270850e+01,
        -8.63375020e+01])

In [309]:
train_set.median().values

array([  5.86450000e-01,   9.50000000e+03,   2.79107766e+04,
         2.00000000e+00,   5.00000000e+00,   3.88270850e+01,
        -8.63375020e+01])

Now we could use this "trained" imputer to transform the training set by replacing missing values by the learned medians

In [310]:
X = imputer.transform(train_set)

The result is a plain Numpy array containing the transformed features. We could put it back into a Pandas DataFrame:

In [311]:
train_set = pd.DataFrame(X, columns=train_set.columns)
college = train_set.drop("PCTFLOAN", axis=1)
college_labels = train_set["PCTFLOAN"].copy()

### Handling text and categorical attributes

Because in our dataset, we don't have categorical varibles and text variables, this step is missing. However, it is possible to add some ategorical varibles into our model.

### Feature scaling

With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes ahve very different scales.

There are two common ways to get all attributes to have the same scale: *min-max scaling* and *standardization*.

In [317]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

college = num_pipeline.fit_transform(college)
college

array([[-0.31677442, -1.23387349, -0.91263209, -0.74241322,  0.70286914,
         0.47568867],
       [ 1.59809777,  2.81318477,  0.61677746, -1.6647419 ,  1.13686701,
         1.20561912],
       [ 0.18713932,  0.41886793, -0.91263209,  1.10224415,  0.39704428,
        -0.8885581 ],
       ..., 
       [-0.31677442, -0.87718482, -0.91263209,  0.17991546, -1.54229203,
         0.46544736],
       [ 0.18804636, -0.61278653, -1.67733686,  0.17991546,  0.23389457,
         0.22678471],
       [ 1.32759688, -0.47476959,  0.61677746,  1.56340849, -0.83118824,
        -1.61640004]])

### Select and train a model

#### Traning and evaluating on the training set

Let's first train a Linear Regression model:

In [318]:
print(college.shape)
print(college_labels.shape)

(6162, 6)
(6162,)


Convert college into pd.DataFrame to get the iloc function work:

In [319]:
college = pd.DataFrame(college)

In [322]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(college, college_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [323]:
some_data = college.iloc[:5]
some_labels = college_labels.iloc[:5]
some_data_prepared = num_pipeline.fit_transform(some_data)
print("Predictions:\t", lin_reg.predict(some_data_prepared))
print("Labels:\t\t", list(some_labels))

Predictions:	 [ 0.58838722  0.67669509  0.52930589  0.52147398  0.32854239]
Labels:		 [0.0, 0.20949999999999999, 0.434, 0.67400000000000004, 0.0088999999999999999]


It works, although the predictions are not exactly accurate. Let's measure this regression model's RMSE on the whole training set using Scikit_Learn's mean_squared_error functions.

In [324]:
from sklearn.metrics import mean_squared_error
college_predictions = lin_reg.predict(college)
lin_mse = mean_squared_error(college_labels, college_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.24897494179336416

The *PCTFLOAN* ranges from 0 to 2, so a typical prediction error of 0.24897494179336416 is not very satisfying.

We may fit the data with a more powerful model, to feed toe training algorithm with better features or to reduce the constraints on the model.

Let's train a DecisionTreeRegressor:

In [325]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(college, college_labels)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

Now the model is trained, let's evaluate it on the training set:

In [326]:
college_predictions = tree_reg.predict(college)
tree_mse = mean_squared_error(college_labels, college_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

1.8570287191493307e-05

The error term is really small. Could this model really be absolutely perfect? of course it is much more likely that the model has badly overfit the data. we could fix that by using part of the training set for training, and part for model validation.

### Better evaluation using cross validation